In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.core import *

import torchtext
from torchtext import vocab, data

from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai import rnn_train, rnn_reg, lm_rnn
from fastai.lm_rnn import *
from fastai.model import fit
from fastai.dataset import *

## Language modeling

In [5]:
PATH='data/wikitext-2/'

In [6]:
%ls {PATH}

wiki.test.tokens  wiki.train.tokens  wiki.valid.tokens


In [7]:
!head -5 {PATH}wiki.train.tokens

 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more <unk> for series newcomers . Char

In [8]:
!wc -lwc {PATH}wiki.train.tokens

   36718  2051910 10797148 data/wikitext-2/wiki.train.tokens


In [9]:
!wc -lwc {PATH}wiki.valid.tokens

   3760  213886 1121681 data/wikitext-2/wiki.valid.tokens


In [10]:
TEXT = data.Field(lower=True)

In [11]:
(trn,val,test) = language_modeling.WikiText2.splits(TEXT, 'data')

In [12]:
TEXT.build_vocab(trn, min_freq=10)
nt = len(TEXT.vocab); nt

12981

In [13]:
len(trn), len(trn[0].text)

(1, 2088628)

In [14]:
trn[0].text[:20]

['<eos>',
 '=',
 'valkyria',
 'chronicles',
 'iii',
 '=',
 '<eos>',
 '<eos>',
 'senjō',
 'no',
 'valkyria',
 '3',
 ':',
 '<unk>',
 'chronicles',
 '(',
 'japanese',
 ':',
 '戦場のヴァルキュリア3',
 ',']

In [15]:
em_sz = 100
nh = 400
nl = 3
bs = 80
bptt = 70

In [16]:
class LanguageModelLoader():
    def __init__(self, ds, bs, bptt):
        text = sum([o.text for o in ds], [])
        fld = ds.fields['text']
        fld.eos_token=False
        nums = fld.numericalize([text])
        self.data = batchify(nums, bs)
        self.bptt = bptt
        self.i = 0
        self.n = len(self.data)
        
    def __iter__(self):
        self.i=0
        return self
    
    def __len__(self): return self.n // self.bptt
        
    def __next__(self):
        bptt = self.bptt if np.random.random() < 0.95 else self.bptt / 2.
        seq_len = max(5, int(np.random.normal(bptt, 5)))
        res = get_batch(self.data, self.i, seq_len)
        self.i += seq_len
        if self.i > self.n-2: self.i=0
        return res

In [26]:
m = Seq2SeqRNN(nt, em_sz, nh, nl, dropout=0.4, dropouth=0.3, dropouti=0.5, dropoute=0.1, wdrop=0.5).cuda()

In [27]:
sl = LanguageModelLoader(trn, bs, bptt)
sl_val = LanguageModelLoader(val, bs, bptt)
md = ModelData({PATH}, sl, sl_val)
len(sl)

372

In [28]:
next(sl)

(Variable containing:
      9    409     35  ...      66   1129      2
     11      7   1538  ...     112     14   1919
   3875      2    267  ...       4   2761   5896
         ...            ⋱           ...         
   3875     11  12472  ...      21    414   3479
     98      0      0  ...      28   3668      4
      4     11     22  ...    3955    792      9
 [torch.cuda.LongTensor of size 74x80 (GPU 0)], Variable containing:
     11
      7
   1538
   ⋮   
    545
   5159
      2
 [torch.cuda.LongTensor of size 5920 (GPU 0)])

In [29]:
opt = optim.SGD(m.parameters(), 30, weight_decay=1e-6)

In [30]:
stepper=get_rnn_stepper(bs)

In [32]:
fit(m, md, 1, nn.CrossEntropyLoss, opt, clip=0.25, stepper_fn=stepper)

A Jupyter Widget



  0%|          | 0/372 [00:00<?, ?it/s]


/home/jhoward/anaconda3/lib/python3.6/site-packages/torch/nn/modules/module.py:224: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greately increasing memory usage. To compact weights again call flatten_parameters().
  result = self.forward(*input, **kwargs)


RuntimeError: invalid argument 2: size '[5520]' is invalid for input of with 71655120 elements at /opt/conda/conda-bld/pytorch_1503970438496/work/torch/lib/TH/THStorage.c:41

### End